<a href="https://colab.research.google.com/github/Varchita-Lalwani/hello-world/blob/master/simple_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
if 'COLAB_TPU_ADDR' not in os.environ: 
  print('Not connected to TPU') 
else: 
  print("Connected to TPU")

Connected to TPU


In [7]:
import tensorflow as tf
import pandas as pd
train_file = pd.read_csv("all.csv")
#train_data = train_file.iloc[0:100,1:2]
#train_labels = train_file.iloc[0:100,4:5]
train_data = train_file.iloc[:,1:2]
train_labels = train_file.iloc[:,4:5]
print(train_labels.shape)





(573, 1)


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_data, train_labels, test_size=0.1, random_state = 42)
dfDummies = pd.get_dummies(Y_train, prefix = 'category')
labels = np.array(dfDummies)
labels.shape

(515, 3)

In [9]:
import re
import nltk
nltk.download('punkt')
length = []
tokenized_sent = []
for each in range(len(X_train)):
  item = X_train.iloc[each,:]
  item = item.values.tolist()
  temp = str(item)
  temp_1 = (re.sub('[^A-Za-z0-9]+', ' ', temp))
  tokens = nltk.word_tokenize(temp_1)
  length.append(len(tokens))
  tokenized_sent.append(tokens)
max_length = max(length)
print(max_length)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
5067


In [0]:
from gensim.models import Word2Vec
vectors = []
model = Word2Vec(tokenized_sent, min_count=1, size=300)
for each in tokenized_sent:
  sent_vect = []
  for every in each:
    sent_vect.append(model.wv[every])
  vectors.append(sent_vect)



In [11]:
from keras.preprocessing import sequence
padded_train_vectors = sequence.pad_sequences(vectors, maxlen=max_length,dtype='float')
padded_train_vectors.shape


Using TensorFlow backend.


(515, 5067, 300)

In [0]:
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.recurrent import SimpleRNN
#from keras import backend as K
model = Sequential()
model.add(SimpleRNN(300, input_shape = (5067,300)))
model.add(Dense(labels.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(padded_train_vectors, labels, epochs=10, steps_per_epoch=2)
y = model.predict(padded_train_vectors)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 300)               180300    
_________________________________________________________________
dense (Dense)                (None, 3)                 903       
Total params: 181,203
Trainable params: 181,203
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10


# New Section